<a href="https://colab.research.google.com/github/ttpro1995/colab_paper_explore/blob/master/spn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import Conv2D, MaxPool2D
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, concatenate
print(tf.VERSION)
print(tf.keras.__version__)
import numpy as np

1.13.1
2.2.4-tf


In [0]:
input_image = np.random.rand(1, 720, 720, 3)  # 720x720 image with 3 channel

In [0]:
# Scale Pyramid Network for Crowd Counting
# Xinya Chen, Yanrui Bin, Nong Sang, Changxin Gao

In [0]:
### first part of SPN before SPM 
SPN_1 = tf.keras.Sequential()
# 2 x cr(64,3)
SPN_1.add(Conv2D(64, kernel_size=3, activation='relu', padding='same'))
SPN_1.add(Conv2D(64, kernel_size=3, activation='relu', padding='same'))
# m
SPN_1.add(MaxPool2D(pool_size=(2, 2)))
# 2 x CR(128,3) 
SPN_1.add(Conv2D(128, kernel_size=3, activation='relu', padding='same'))
SPN_1.add(Conv2D(128, kernel_size=3, activation='relu', padding='same'))
# m
SPN_1.add(MaxPool2D(pool_size=(2, 2)))
# 3 x CR(256,3)
SPN_1.add(Conv2D(256, kernel_size=3, activation='relu', padding='same'))
SPN_1.add(Conv2D(256, kernel_size=3, activation='relu', padding='same'))
SPN_1.add(Conv2D(256, kernel_size=3, activation='relu', padding='same'))
# m
SPN_1.add(MaxPool2D(pool_size=(2, 2)))
# 3 x CR(512,3) 
SPN_1.add(Conv2D(512, kernel_size=3, activation='relu', padding='same'))
SPN_1.add(Conv2D(512, kernel_size=3, activation='relu', padding='same'))
SPN_1.add(Conv2D(512, kernel_size=3, activation='relu', padding='same'))

In [7]:
out_1 = SPN_1.predict(input_image)
out_1.shape

(1, 90, 90, 512)

In [0]:
spm_input = Input([None, None, 512])
# conv 3 - n - 2 with n = 512 
conv_3_n_2 = Conv2D(512, kernel_size=3, activation='relu', padding='same', dilation_rate=2)(spm_input)
# conv 3 - n - 4
conv_3_n_4 = Conv2D(512, kernel_size=3, activation='relu', padding='same', dilation_rate=4)(spm_input)
# conv 3 - n - 8
conv_3_n_8 = Conv2D(512, kernel_size=3, activation='relu', padding='same', dilation_rate=8)(spm_input)
# conv 3 - n - 12
conv_3_n_12 = Conv2D(512, kernel_size=3, activation='relu', padding='same', dilation_rate=12)(spm_input)
concat_x = concatenate([conv_3_n_2, conv_3_n_4, conv_3_n_8, conv_3_n_12],  axis=3)
SPM =  Model(spm_input, concat_x)

In [26]:
out_1.shape

(1, 90, 90, 512)

In [27]:
out_2 = SPM.predict(out_1)
out_2.shape

(1, 90, 90, 2048)

In [0]:
### full SPN network 

SPN = tf.keras.Sequential()
# 2 x cr(64,3)
SPN.add(Conv2D(64, kernel_size=3, activation='relu', padding='same'))
SPN.add(Conv2D(64, kernel_size=3, activation='relu', padding='same'))
# m
SPN.add(MaxPool2D(pool_size=(2, 2)))
# 2 x CR(128,3) 
SPN.add(Conv2D(128, kernel_size=3, activation='relu', padding='same'))
SPN.add(Conv2D(128, kernel_size=3, activation='relu', padding='same'))
# m
SPN.add(MaxPool2D(pool_size=(2, 2)))
# 3 x CR(256,3)
SPN.add(Conv2D(256, kernel_size=3, activation='relu', padding='same'))
SPN.add(Conv2D(256, kernel_size=3, activation='relu', padding='same'))
SPN.add(Conv2D(256, kernel_size=3, activation='relu', padding='same'))
# m
SPN.add(MaxPool2D(pool_size=(2, 2)))
# 3 x CR(512,3) 
SPN.add(Conv2D(512, kernel_size=3, activation='relu', padding='same'))
SPN.add(Conv2D(512, kernel_size=3, activation='relu', padding='same'))
SPN.add(Conv2D(512, kernel_size=3, activation='relu', padding='same'))
## 
# TODO: SPM
spm_input = Input([None, None, 512])
# conv 3 - n - 2 with n = 512 
conv_3_n_2 = Conv2D(512, kernel_size=3, activation='relu', padding='same', dilation_rate=2)(spm_input)
# conv 3 - n - 4
conv_3_n_4 = Conv2D(512, kernel_size=3, activation='relu', padding='same', dilation_rate=4)(spm_input)
# conv 3 - n - 8
conv_3_n_8 = Conv2D(512, kernel_size=3, activation='relu', padding='same', dilation_rate=8)(spm_input)
# conv 3 - n - 12
conv_3_n_12 = Conv2D(512, kernel_size=3, activation='relu', padding='same', dilation_rate=12)(spm_input)
concat_x = concatenate([conv_3_n_2, conv_3_n_4, conv_3_n_8, conv_3_n_12],  axis=3)
SPM =  Model(spm_input, concat_x)
SPN.add(SPM)
# 3 x CR(512, 3)
SPN.add(Conv2D(512, kernel_size=3, activation='relu', padding='same'))
SPN.add(Conv2D(512, kernel_size=3, activation='relu', padding='same'))
SPN.add(Conv2D(512, kernel_size=3, activation='relu', padding='same'))

# 1x CR(256,3) 
SPN.add(Conv2D(256, kernel_size=3, activation='relu', padding='same'))

# 1 x CR(1,1)
SPN.add(Conv2D(1, kernel_size=1, activation='relu', padding='same'))

In [29]:
out_full_spn = SPN.predict(input_image)
out_full_spn.shape

(1, 90, 90, 1)

In [30]:
720/90

8.0